In [1]:
import pandas as pd
import sqlalchemy
import requests
import re
from io   import StringIO
from bs4  import BeautifulSoup as BS
from time import sleep
import datetime as dt

In [2]:
import os

In [3]:
import warnings

## Custom Function Imports

In [4]:
from functions.logit import load_downloaded
from functions.status import print_refresh

In [5]:
from functions.df_func import low_price

In [6]:
## Important Constant Variables

In [7]:
# the Start and End Dates to look for Final Fantasy Prices
start_date = pd.to_datetime('2025-06-09', format='%Y-%m-%d')
end_date = pd.to_datetime('2025-06-15', format='%Y-%m-%d')

In [8]:
URL = 'https://www.mtggoldfish.com/tournament/pro-tour-final-fantasy#paper'

response = requests.get(URL)
response.status_code

sleep(3)


In [9]:
'''
URL5 = 'https://www.mtggoldfish.com/tournament/pro-tour-final-fantasy?page=5'

response5 = requests.get(URL)
response5.status_code

sleep(30)
'''

"\nURL5 = 'https://www.mtggoldfish.com/tournament/pro-tour-final-fantasy?page=5'\n\nresponse5 = requests.get(URL)\nresponse5.status_code\n\nsleep(30)\n"

In [10]:
soup = BS(response.text)
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html data-skip-ads="false" data-theme="light" lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-PS4ZC64YQQ">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());
  gtag('config', 'G-PS4ZC64YQQ');
  </script>
  <title>
   Pro Tour Final Fantasy (Standard) Decks
  </title>
  <meta content="Pro Tour Final Fantasy (Standard) Magic: the Gathering deck lists with prices." name="description"/>
  <meta content="pro tour final fantasy, standard, results, deck, decks, deck list, tournament, daily event, finish, top 8, magic, mtg, magic the gathering, magic the gathering online, mtgo, mtg arena, arena, tabletop, daily events" name="keywords"/>
  <meta content="Pro Tour Final Fantasy (Standard) Decks" property="og:title"/>
  <meta content="websit

In [11]:
tournament = soup.find('table', attr={'class' : 'table-tournament'})

In [12]:
print(soup.find('table', attrs={'class' : 'table-tournament'}).prettify())

<table class="table-tournament">
 <thead>
  <tr>
   <th>
    Place
   </th>
   <th>
    Deck
   </th>
   <th>
    Pilot
   </th>
   <th class="responsive-column text-right">
    Tabletop Price
   </th>
   <th class="responsive-column text-right">
    MTGO Price
   </th>
   <th class="text-right">
    Toggle Deck
   </th>
  </tr>
 </thead>
 <tr class="tournament-decklist-odd">
  <td class="text-right">
   1st
  </td>
  <td>
   <a href="/deck/7186307">
    Mono-Red Aggro
   </a>
   <span aria-label="mana cost: red" class="manacost">
    <i class="ms ms-r ms-cost ms-shadow">
    </i>
   </span>
  </td>
  <td>
   <a href="/player/Ken+Yukuhiro">
    Ken Yukuhiro
   </a>
  </td>
  <td class="responsive-column text-right">
   $ 134
  </td>
  <td class="responsive-column text-right">
   39 tix
  </td>
  <td class="text-right tournament-decklist-toggle" data-deckid="7186307">
   <a href="javascript:expand_deck(7186307)">
    Expand
   </a>
  </td>
 </tr>
 <tr class="tournament-decklist" data-de

In [13]:
tournament_df = pd.read_html(
    StringIO(
        str(
            soup.find('table', attrs = {'class' : 'table-tournament'})
        )
    )
)[0]

In [14]:
tournament_df

,Place,Deck,Pilot,Tabletop Price,MTGO Price,Toggle Deck
0,1st,Mono-Red Aggro,Ken Yukuhiro,$ 134,39 tix,Expand
1,2nd,Izzet Prowess,Ian Robb,$ 428,237 tix,Expand
2,4th,Mono-Red Aggro,Yuchen Liu,$ 135,39 tix,Expand
3,4th,Izzet Prowess,Toni Portolan,$ 389,240 tix,Expand
4,8th,Mono-Red Aggro,Percy Fang,$ 151,50 tix,Expand
...,...,...,...,...,...,...
95,6 - 4,Izzet Prowess,Matt Costa,$ 361,226 tix,Expand
96,6 - 4,Izzet Prowess,Matt Sperling,$ 313,210 tix,Expand
97,6 - 4,Mono-Red Aggro,Pedro Perrini,$ 133,39 tix,Expand
98,6 - 4,Izzet Prowess,Raja Sulaiman,$ 379,236 tix,Expand


In [15]:
decks_table = soup.find('table')
deck_rows = decks_table.findAll('tr')
all_links = [deck.findAll('a') for deck in deck_rows[1:-1:2]] # Step by 2 because every other row was returning blank.
for link in all_links:
  print(link, '\n')
deck_links = [link[0].get('href') for link in all_links]
print(deck_links)
print(len(deck_links))
#deck_links = [deck.get('href') for row in
#for links in all_links[0::2]:
#    print(link, '\n')
#print(deck_links[0][0].get('href'))

[<a href="/deck/7186307">Mono-Red Aggro</a>, <a href="/player/Ken+Yukuhiro">Ken Yukuhiro</a>, <a href="javascript:expand_deck(7186307)">Expand</a>] 

[<a href="/deck/7186234">Izzet Prowess</a>, <a href="/player/Ian+Robb">Ian Robb</a>, <a href="javascript:expand_deck(7186234)">Expand</a>] 

[<a href="/deck/7186164">Mono-Red Aggro</a>, <a href="/player/Yuchen+Liu">Yuchen Liu</a>, <a href="javascript:expand_deck(7186164)">Expand</a>] 

[<a href="/deck/7186223">Izzet Prowess</a>, <a href="/player/Toni+Portolan">Toni Portolan</a>, <a href="javascript:expand_deck(7186223)">Expand</a>] 

[<a href="/deck/7186066">Mono-Red Aggro</a>, <a href="/player/Percy+Fang">Percy Fang</a>, <a href="javascript:expand_deck(7186066)">Expand</a>] 

[<a href="/deck/7185985">Izzet Prowess</a>, <a href="/player/Christian+Baker">Christian Baker</a>, <a href="javascript:expand_deck(7185985)">Expand</a>] 

[<a href="/deck/7186238">Izzet Prowess</a>, <a href="/player/David+Rood">David Rood</a>, <a href="javascript:ex

In [16]:
tournament_df.head()

,Place,Deck,Pilot,Tabletop Price,MTGO Price,Toggle Deck
0,1st,Mono-Red Aggro,Ken Yukuhiro,$ 134,39 tix,Expand
1,2nd,Izzet Prowess,Ian Robb,$ 428,237 tix,Expand
2,4th,Mono-Red Aggro,Yuchen Liu,$ 135,39 tix,Expand
3,4th,Izzet Prowess,Toni Portolan,$ 389,240 tix,Expand
4,8th,Mono-Red Aggro,Percy Fang,$ 151,50 tix,Expand


In [17]:
with open('../protour/final-fantasy/page1.htm') as f:
    soup_expanded = BS( f.read() )


In [18]:
print(
    soup_expanded.find('div', attrs={'class':'deck-table-container'} )  
)

<div class="deck-table-container">
<table class="deck-view-deck-table">
<tbody><tr class="deck-category-header">
<th colspan="4">
Creatures
(28)
</th>
</tr>
<tr>
<td class="text-right">
4
</td>
<td>
<span class="card_id card_name"><a data-card-id="Heartfire Hero [BLB]" data-full-image="https://cdn1.mtggoldfish.com/images/gf/Heartfire%2BHero%2B%255BBLB%255D.jpg" data-full-image-rotation="FALSE" data-original-title="" href="https://www.mtggoldfish.com/price/Bloomburrow/Heartfire+Hero#paper" rel="popover" title="">Heartfire Hero</a></span>
</td>
<td>
<span aria-label="mana cost: red" class="manacost"><i class="ms ms-r ms-cost ms-shadow"></i></span>
</td>
<td class="text-right">
$ 2.16
</td>
</tr>
<tr>
<td class="text-right">
4
</td>
<td>
<span class="card_id card_name"><a data-card-id="Hired Claw &lt;showcase&gt; [BLB]" data-full-image="https://cdn1.mtggoldfish.com/images/gf/Hired%2BClaw%2B%253Cshowcase%253E%2B%255BBLB%255D.jpg" data-full-image-rotation="FALSE" data-original-title="" href

In [44]:
decks_df_list = pd.read_html(
    StringIO(
        str(
            soup_expanded.findAll('div', attrs={'class':'deck-table-container'} )
        )
    )
)

In [20]:
deck_df2 = pd.read_html(
    StringIO(
        str(
            soup_expanded.find('div', attrs={'class':'deck-table-container'} )
        )
    )
)

## Make deck DataFrames

In [173]:
decks_df = []

for number in range(1, 5):
  with open(f'../protour/final-fantasy/page{number}.htm') as f:
    soup_expanded = BS( f.read() )

  temp_list = pd.read_html(
    StringIO(
      str(
        soup_expanded.findAll('div', attrs={'class':'deck-table-container'} )
      )
    )
  )

  n = len( temp_list )
  for i in range(0, n, 2):
    print_refresh(f'{i//2}')
    
    standard_cols = dict(zip(decks_df_list[i].columns, ['count', 'card name', 'N/A', 'current price']))
    decks_df_list[i] = decks_df_list[i].rename( columns = standard_cols )

    standard_cols = dict(zip(decks_df_list[i+1].columns, ['count', 'card name', 'N/A', 'current price']))
    decks_df_list[i+1] = decks_df_list[i+1].rename( columns = standard_cols )

    temp_df = pd.concat( [ decks_df_list[i], decks_df_list[i+1] ] )[ ['card name', 'count'] ]   
    temp_df = temp_df.loc[ temp_df['card name'] != temp_df['count'] ]
    decks_df.append( temp_df )

In [ ]:
with open(

In [83]:
decks_df[99]

,card name,count
0,Heartfire Hero,4
1,Hired Claw,4
2,Emberheart Challenger,4
3,Magebane Lizard,3
4,Manifold Mouse,4
5,Screaming Nemesis,4
7,Burst Lightning,4
8,Monstrous Rage,4
9,Opera Love Song,3
10,Witchstalker Frenzy,4


In [23]:
''' Location Moved

manifold_mouse = pd.read_csv(
    '../protour/final-fantasy/Manifold Mouse [BLB].csv',
    names = [ 'date' , 'price' ]
)
'''

" Location Moved\n\nmanifold_mouse = pd.read_csv(\n    '../protour/final-fantasy/Manifold Mouse [BLB].csv',\n    names = [ 'date' , 'price' ]\n)\n"

In [24]:
# deck_df['manifold_mouse']['date']

In [25]:
###
''' '''
'''

end_date = pd.to_datetime('2025-06-15', format='%Y-%m-%d')
start_date = pd.to_datetime('2025-06-09', format='%Y-%m-%d')

manifold_mouse['date'] = pd.to_datetime(manifold_mouse['date'])

temp = manifold_mouse.loc[
  manifold_mouse['date'].between(start_date, end_date, inclusive = 'both')                         
]

temp.nsmallest(1, 'price')

#for index, row in manifold_mouse.iterrows():
#  if start_date <= row['date'] and row['date'] <= end_date:
#    if minimum > row['price']:
#      minimum = row['price']
#    print(row['price'])

'''

"\n\nend_date = pd.to_datetime('2025-06-15', format='%Y-%m-%d')\nstart_date = pd.to_datetime('2025-06-09', format='%Y-%m-%d')\n\nmanifold_mouse['date'] = pd.to_datetime(manifold_mouse['date'])\n\ntemp = manifold_mouse.loc[\n  manifold_mouse['date'].between(start_date, end_date, inclusive = 'both')                         \n]\n\ntemp.nsmallest(1, 'price')\n\n#for index, row in manifold_mouse.iterrows():\n#  if start_date <= row['date'] and row['date'] <= end_date:\n#    if minimum > row['price']:\n#      minimum = row['price']\n#    print(row['price'])\n\n"

## Function Tests

## CSV Price Dictionary

In [26]:
price_csv_dict = {}
card_list = load_downloaded()
csv_list = os.listdir('../cardprices')

if len(card_list) == len(csv_list):
  print(f'Things are looking good.')
else:
  print( f'{len(card_list)} != {len(csv_list)}, missing card names or CSVs' )

for index, card_name in enumerate(card_list):
  card_name        = re.sub( r"[/:]", '_', card_name, count=0)
  card_list[index] = card_name.strip().strip("'").strip()

print(csv_list[0])
print(card_list[0])

for csv in csv_list:
  print_refresh(f'Looking for {csv} match: {len(price_csv_dict)} csv files found.              ')
  for card in card_list:
    if card.lower().strip().strip("'") == csv[:len( card.strip().strip("'") )].lower():
      price_csv_dict[card] = csv
      print_refresh(f'{csv} match found: {len(price_csv_dict)} csv files found.                     ')
      sleep(.01)

print( f'{ price_csv_dict[ card_list[0] ] }    ' )
price_csv_dict['Opt'] = 'Opt _beginner box_ [FDN].csv'


if len(card_list) == len(price_csv_dict):
  print( f'Seems good: \nCard List is {len(card_list)} long. \nName-CSV Dictionary is {len(price_csv_dict)}' )
else:
  print( f'Something went wrong, \n{len(card_list)} != {len(price_csv_dict)}' )

402 != 403, missing card names or CSVs
Abhorrent Oculus [DSK].csv
Heartfire Hero
Heartfire Hero [BLB].csv    s match: 402 csv files found.               :::s found.                      ::::                    :::
Seems good: 
Card List is 402 long. 
Name-CSV Dictionary is 402


In [27]:
for csv_name in csv_list:
  if csv_name not in price_csv_dict.values():
    print(csv_name)
price_csv_dict['Opt']
price_csv_dict['Optimistic Scavenger']

[EOC] Useless


'Optimistic Scavenger _Monster_ [DSK].csv'

In [28]:
test_string = f'../cardprices/{price_csv_dict[card_list[0]]}'
test_df = pd.read_csv(test_string)

In [29]:
test_df = pd.read_csv( '../cardprices/Torch the Tower [WOE].csv' )
test_df.head()

,2023-08-15,1.5
0,2023-08-16,1.36
1,2023-08-17,1.30
2,2023-08-18,1.24
3,2023-08-19,1.16
4,2023-08-20,1.13


In [30]:
test_df.head()

,2023-08-15,1.5
0,2023-08-16,1.36
1,2023-08-17,1.30
2,2023-08-18,1.24
3,2023-08-19,1.16
4,2023-08-20,1.13


In [31]:
print( card_list[0:5] )
print( price_csv_dict[ card_list[0] ] )

['Heartfire Hero', 'Hired Claw', 'Emberheart Challenger', 'Magebane Lizard', 'Manifold Mouse']
Heartfire Hero [BLB].csv


## Low Prices DataFrame

In [32]:
low_price_df = pd.DataFrame()

# warnings.filterwarnings('ignore')
i=0
for card_name, csv_file in price_csv_dict.items():
  csv_path = f'../cardprices/{csv_file}'

  prices_df = pd.read_csv( csv_path, names = ['date', 'price'] )
  prices_df.head()
  card_row = low_price(card_name.strip().strip("'"), prices_df, start_date, end_date)
  low_price_df = pd.concat( [low_price_df, card_row] )

# warnings.resetwarnings()

In [33]:
low_price_df.reset_index(drop=True, inplace=True)

In [34]:
len(price_csv_dict)

402

In [35]:
low_price_df.head()

,card name,low price
0,Abhorrent Oculus,17.07
1,Abrade,0.20
2,Abuelo's Awakening,0.30
3,"Aclazotz, Deepest Betrayal",4.27
4,Adarkar Wastes,0.75


In [36]:
cards_in_df_list = []
for index, row in low_price_df.iterrows():
    cards_in_df_list.append(row['card name'])

In [37]:
for card in price_csv_dict:
  if card not in cards_in_df_list:
    print(card)

print(f'If this is the only thing that prints, it is a good sign')

If this is the only thing that prints, it is a good sign


In [38]:
for index, card_name in enumerate(card_list):
  if card_name not in price_csv_dict.keys():
    print(index, card_name)

In [39]:
for index, card_name in enumerate(card_list):
  card_list[index] = re.sub( r"[/:]", '_', card_name, count=0)

print(card_list[48])
print(card_list[153])

Roaring Furnace __ Steaming Sauna
Sidequest_ Play Blitzball


In [40]:
low_price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   card name  402 non-null    object 
 1   low price  402 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.4+ KB


In [41]:
''' File In Place does not need to be overwritten '''
# low_price_df.to_csv('./finished/final-fantasy-card-prices.csv')

' File In Place does not need to be overwritten '

In [42]:
# re.sub(r'[/:]', '_', 'Fire // Ice' )

'Fire __ Ice'

## Processs Entire Tournament

In [113]:
tournament_df = pd.DataFrame()

for i in range(4):
  with open(f'../protour/final-fantasy/colapsed{i+1}.htm') as f:
    soup_expanded = BS( f.read() )

  symbols = soup_expanded.findAll('span', attrs = {'class' : 'manacost' })
  for index, symbol in enumerate(symbols):
    symbols[index] = re.sub('mana cost: ', '', symbol['aria-label'])

  colors_df = pd.DataFrame( {'Colors' : symbols} )
    
  temp_df = pd.read_html(
    StringIO(
      str(
        soup_expanded.find('table', attrs = {'class' : 'table-tournament'})
       )
    )
  )[0]
    
  temp_df = pd.concat( [temp_df, colors_df], axis=1 )
    
  tournament_df = pd.concat( [ tournament_df, temp_df ] )

tournament_df

,Place,Deck,Pilot,Tabletop Price,MTGO Price,Toggle Deck,Colors
0,1st,Mono-Red Aggro,Ken Yukuhiro,$ 134,39 tix,Expand,red
1,2nd,Izzet Prowess,Ian Robb,$ 428,237 tix,Expand,blue red
2,4th,Mono-Red Aggro,Yuchen Liu,$ 135,39 tix,Expand,red
3,4th,Izzet Prowess,Toni Portolan,$ 389,240 tix,Expand,blue red
4,8th,Mono-Red Aggro,Percy Fang,$ 151,50 tix,Expand,red
...,...,...,...,...,...,...,...
24,-,Orzhov Pixie,Matthew Baros-Steyl,$ 198,43 tix,Expand,white black
25,-,Jund Midrange,PÃ¤r Jones,$ 385,155 tix,Expand,black red green
26,-,Boros Aggro,Yoshihiko Ikawa,$ 139,40 tix,Expand,red
27,-,Dimir Midrange,Jean-Emmanuel Depraz,$ 252,64 tix,Expand,blue black


In [114]:
tournament_df = tournament_df.rename( columns = {'Tabletop Price': 'Current Price'} )
tournament_df = tournament_df[['Place', 'Deck', 'Colors', 'Pilot', 'Current Price']]

In [115]:
tournament_df.reset_index( drop=True, inplace=True )
tournament_df

,Place,Deck,Colors,Pilot,Current Price
0,1st,Mono-Red Aggro,red,Ken Yukuhiro,$ 134
1,2nd,Izzet Prowess,blue red,Ian Robb,$ 428
2,4th,Mono-Red Aggro,red,Yuchen Liu,$ 135
3,4th,Izzet Prowess,blue red,Toni Portolan,$ 389
4,8th,Mono-Red Aggro,red,Percy Fang,$ 151
...,...,...,...,...,...
324,-,Orzhov Pixie,white black,Matthew Baros-Steyl,$ 198
325,-,Jund Midrange,black red green,PÃ¤r Jones,$ 385
326,-,Boros Aggro,red,Yoshihiko Ikawa,$ 139
327,-,Dimir Midrange,blue black,Jean-Emmanuel Depraz,$ 252


## Calculate Deck Prices

In [174]:
deck_price_list = []

for number, deck in enumerate(decks_df):
  deck = deck[['card name', 'count' ]] # gets rid of previous low cost column
  deck = deck.merge(low_price_df, on='card name')
  deck['Total Cost'] = ''

  warnings.simplefilter('ignore')
  for index, row in deck.iterrows():
    deck['Total Cost'][index] = round(
      int(row['count']) * row['low price'],
      2
    )
      
  deck_price_list.append( round(deck['Total Cost'].sum(), 2) )
  decks_df[number] = deck
    
  warnings.resetwarnings()

In [180]:
temp_df = pd.DataFrame( {'Estimated Price' : deck_price_list} )
tournament_prices_df = pd.concat(
  [ tournament_df, temp_df ], axis = 1
)

tournament_prices_df

,Place,Deck,Colors,Pilot,Current Price,Estimated Price
0,1st,Mono-Red Aggro,red,Ken Yukuhiro,$ 134,177.47
1,2nd,Izzet Prowess,blue red,Ian Robb,$ 428,461.49
2,4th,Mono-Red Aggro,red,Yuchen Liu,$ 135,180.06
3,4th,Izzet Prowess,blue red,Toni Portolan,$ 389,418.61
4,8th,Mono-Red Aggro,red,Percy Fang,$ 151,195.62
...,...,...,...,...,...,...
324,-,Orzhov Pixie,white black,Matthew Baros-Steyl,$ 198,176.20
325,-,Jund Midrange,black red green,PÃ¤r Jones,$ 385,207.91
326,-,Boros Aggro,red,Yoshihiko Ikawa,$ 139,398.25
327,-,Dimir Midrange,blue black,Jean-Emmanuel Depraz,$ 252,309.54


In [183]:
# Already Written

# tournament_prices_df.to_csv('./finished/final-fantasy-deck-prices.csv')

## Process Other Tournaments

In [186]:
events_list = [
    {'aetherdrift'           : 4},
    {'thunder-junction'      : 3},
    {'march-of-the-machine'  : 3}
]
    

## Grab all Decks from each event

In [211]:
decked_events = []

In [212]:
decks_dfs = {
    'aetherdrift'           : [],
    'thunder-junction'      : [],
    'march-of-the-machine'  : []    
}

for dictionary in events_list:
  for event, pages in dictionary.items():
    if event in decked_events:
      continue
        
    for number in range(pages):
      with open(f'../protour/{event}/open{number+1}.htm') as f:
        soup_expanded = BS( f.read() )

      print_refresh(f'Processing decks for {event} page {number+1}')
      temp_list = pd.read_html(
        StringIO(
          str(
            soup_expanded.findAll('div', attrs={'class':'deck-table-container'} )
          )
        )
      )

      n = len( temp_list )
      for i in range(0, n, 2):
        print_refresh(f'{i//2}')
    
        standard_cols = dict(zip(temp_list[i].columns, ['count', 'card name', 'N/A', 'current price']))
        temp_list[i] = temp_list[i].rename( columns = standard_cols )

        standard_cols = dict(zip(temp_list[i+1].columns, ['count', 'card name', 'N/A', 'current price']))
        temp_list[i+1] = temp_list[i+1].rename( columns = standard_cols )

        temp_df = pd.concat( [ temp_list[i], temp_list[i+1] ] )[ ['card name', 'count'] ]   
        temp_df = temp_df.loc[ temp_df['card name'] != temp_df['count'] ]
        decks_dfs[event].append( temp_df )

    decked_events.append(event)

## Grab Tournament results from each event

In [219]:
grabbed_tournaments = []

In [224]:
tournament_dfs = {}


for dictionary in events_list:
  for event, pages in dictionary.items():
    if event in grabbed_tournaments:
      continue
    else:
      tournament_dfs[event] = pd.DataFrame()
      
    for number in range(pages):
      print_refresh(f'fetching {event} table {number+1}')
      with open(f'../protour/{event}/closed{number+1}.htm') as f:
        soup_expanded = BS( f.read() )

      symbols = soup_expanded.findAll('span', attrs = {'class' : 'manacost' })
      for index, symbol in enumerate(symbols):
        symbols[index] = re.sub('mana cost: ', '', symbol['aria-label'])

      colors_df = pd.DataFrame( {'Colors' : symbols} )
    
      temp_df = pd.read_html(
        StringIO(
          str(
            soup_expanded.find('table', attrs = {'class' : 'table-tournament'})
           )
        )
      )[0]
    
      temp_df = pd.concat( [temp_df, colors_df], axis=1 )
    
      tournament_dfs[event] = pd.concat( [ tournament_dfs[event], temp_df ] )
      tournament_dfs[event].reset_index( drop=True, inplace=True )
      
    print( len(tournament_df) )

329 fetching aetherdrift table 4 :::
329 fetching thunder-junction table 3 :::
329 fetching march-of-the-machine table 3 :::


In [229]:
tournament_dfs['march-of-the-machine']

,Place,Deck,Pilot,Tabletop Price,MTGO Price,Toggle Deck,Colors
0,7th,Rakdos Reanimator,cain_rianhard,$ 314,339 tix,Expand,black red
1,8th,Five-color Ramp,David Olsen,$ 346,333 tix,Expand,white red green
2,11th,Rakdos Midrange,Daniel Goetschel,$ 551,422 tix,Expand,black red
3,12th,Rakdos Midrange,Willy Edel,$ 539,420 tix,Expand,black red
4,13th,Rakdos Midrange,David Inglis,$ 393,366 tix,Expand,black red
...,...,...,...,...,...,...,...
225,248th,Esper Legends,Chris Westerlund,$ 500,247 tix,Expand,white blue black
226,249th,Grixis Midrange,shun asano,$ 488,390 tix,Expand,blue black red
227,250th,Esper Legends,Boston Schatteman,$ 538,242 tix,Expand,white blue black
228,251st,Grixis Midrange,toru kono,$ 424,367 tix,Expand,blue black red


## Low Price Data Frames per Event

In [362]:
format = '%Y-%m-%d'
events_dates = {
    'aetherdrift' : 
      {'start' : '2025-02-07',
       'end'   : '2025-02-14' },
    'thunder-junction': 
      {'start' : '2024-04-12', 'end' : '2024-04-19'},
    'march-of-the-machine' :
      {'start' : '2023-04-21', 'end' : '2023-04-28'}
}

#for dict in events_dates:
#  for key, event in dict.items():
#    event['start'] = pd.to_datetime(event['start'], format=format)


In [408]:
low_price_dfs = {}


for event, dates in events_dates.items():
    event_cards = []
    print()
    print(f'working on {event}')
    names_to_csv = {}
    csv_list = os.listdir('../cardprices')
    price_csv_dict = {}

    print_refresh(f'Fetching {event} card names')
    for deck in decks_dfs[event]:
      for index, row in deck.iterrows():
        if row['card name'] not in event_cards:
          event_cards.append( row['card name'] )

    print_refresh(f'Preparing {event}')
    for index, card_name in enumerate(event_cards):
      card_name        = re.sub( r"[/:]", '_', card_name, count=0)
      event_cards[index] = card_name.strip().strip("'").strip()

    print_refresh(f'Preparing Stage 2 {event}')
    csv_list.sort()
    event_cards.sort()
    #print(event_cards)
    # loops CSV first to catch shorter names i.e. Opt vs Optomistic Scavaging
    for csv in csv_list:
      #print(csv)
      for card in event_cards:
        if re.match(card, csv):
          names_to_csv[card] = csv
          print_refresh(f'{card}: {names_to_csv[card]}')
          continue
            
    print(len(names_to_csv))

    
    low_price_dfs[event] = pd.DataFrame()
    for card in event_cards:
      csv = names_to_csv[card]
      temp_df  = pd.read_csv(f'../cardprices/{csv}', names = ['date', 'price'])
      card_row = low_price( 
          card, temp_df,
          start = dates['start'],
          end   = dates['end']
      )
      if 1 == len(card_row):
        low_price_dfs[event] = pd.concat( [low_price_dfs[event] , card_row] )
      else:
        print(f'Problem w/ {card} csv for {event}')
    print(f'{len(low_price_dfs[event])} of {len(event_cards)} worked in {event}') 
  


working on aetherdrift
276 Zur, Eternal Schemer: Zur, Eternal Schemer _showcase_ [DMU] (F).csv :::::::::ture foil_ [DSK].csv :::
276 of 276 worked in aetherdrift

working on thunder-junction
138 Witness Protection: Witness Protection [SNC].csv :::::.csv :::csv :::v :::: :::
138 of 138 worked in thunder-junction

working on march-of-the-machine
76: Witness Protection: Witness Protection [SNC].csv :::F).csv ::: :::csv ::::::
Problem w/ Abrade csv for march-of-the-machine
75 of 76 worked in march-of-the-machine


In [308]:
os.listdir('../cardprices')[0]

'Abandoned Sarcophagus [C20].csv'

In [354]:
low_price_dfs.keys()

dict_keys(['aetherdrift', 'thunder-junction', 'march-of-the-machine'])

In [357]:
low_price_dfs['march-of-the-machine']

,card name,low price
438,"Adeline, Resplendent Cathar",8.89
251,"Ajani, Sleeper Agent",5.50
22,Alabaster Host Intercessor,0.15
93,Annex Sentry,0.25
94,Anoint with Affliction,0.19
...,...,...
171,Yotian Frontliner,0.24
93,Zenith Chronicler,0.40
169,Zephyr Sentinel,0.20
410,Ziatora's Proving Ground,7.00


In [278]:
decks_dfs['aetherdrift'][0]['card name']

0               Zur, Eternal Schemer
1          Beza, the Bounding Spring
2         Overlord of the Hauntwoods
3          Overlord of the Mistmoors
5                 Analyze the Pollen
6                           Get Lost
7                    Day of Judgment
8                         Ride's End
9                            Sunfall
11                  Up the Beanstalk
12                Temporary Lockdown
13                   Leyline Binding
15                   Floodfarm Verge
16                            Forest
17                        Hedge Maze
18                    Hushwood Verge
0                             Island
1                       Lush Portico
2                             Plains
3                 Razorverge Thicket
4                 Shadowy Backstreet
5                              Swamp
6                    Wastewood Verge
8                    Elspeth's Smite
9                             Negate
10                Pawpatch Formation
11                     Rest in Peace
1

In [ ]:
# price_csv_dict['Opt'] = 'Opt _beginner box_ [FDN].csv'

## Calculate All Deck Prices

In [409]:
deck_price_lists_dict = {}
tournament_prices_dfs  = {}


for event, decks in decks_dfs.items():
  deck_price_lists_dict[event] = []
  for number, deck in enumerate(decks):
    deck = deck[['card name', 'count' ]] # gets rid of previous low cost column
    deck = deck.merge(low_price_df, on='card name')
    deck['Total Cost'] = ''

    warnings.simplefilter('ignore')
    for index, row in deck.iterrows():
      deck['Total Cost'][index] = round(
        int(row['count']) * row['low price'],
        2
      )
      
    deck_price_lists_dict[event].append( round(deck['Total Cost'].sum(), 2) )
    decks_dfs[event][number] = deck
    
    warnings.resetwarnings()

In [375]:
# decks_dfs['aetherdrift'][0]
# deck_price_lists_dict['aetherdrift']

In [410]:
## work on

for event, deck_prices in deck_price_lists_dict.items():
  temp_df = pd.DataFrame( {'Estimated Price' : deck_prices} )
  tournament_prices_dfs[event] = pd.concat(
    [ tournament_dfs[event], temp_df ], axis = 1
  )

  tournament_prices_dfs[event]

In [394]:
#already Written

# tournament_prices_dfs['aetherdrift'].to_csv('./finished/aetherdrift.csv')

In [407]:
#already Written

# tournament_prices_dfs['thunder-junction'].to_csv('./finished/thunder-junction.csv')

In [411]:
#already Written

# tournament_prices_dfs['march-of-the-machine'].to_csv('./finished/march-of-the-machine.csv')

In [412]:
tournament_prices_dfs['thunder-junction']

,Place,Deck,Pilot,Tabletop Price,MTGO Price,Toggle Deck,Colors,Estimated Price
0,1st,Domain Ramp,Yoshihiko Ikawa,$ 499,149 tix,Expand,white blue black green,216.83
1,2nd,Azorius Control,Yuta Takahashi,$ 270,290 tix,Expand,white blue,79.58
2,4th,Boros Convoke,Takumi Matsuura,$ 199,70 tix,Expand,white red,117.69
3,4th,Esper Midrange,Arne Huschenbeth,$ 347,182 tix,Expand,white blue black,117.63
4,8th,Four-Color Legends,Jason Ye,$ 613,246 tix,Expand,blue black red green,197.79
...,...,...,...,...,...,...,...,...
202,-,Boros Convoke,Andy Garcia-Romo,$ 201,71 tix,Expand,white red,155.57
203,-,Domain Ramp,Brian Zilles,$ 522,225 tix,Expand,white blue black green,233.00
204,-,Esper Midrange,Jean-Emmanuel Depraz,$ 485,192 tix,Expand,white blue black,360.83
205,-,Esper Midrange,James Larsen-Scott,$ 364,150 tix,Expand,white blue black,198.25
